In [80]:
import cv2 as cv
import imageio
import numpy as np
import math
import matplotlib.pyplot as plt
from os import listdir
from os import system
import os
import random
import sys
sys.path.append('PythonSIFT/')
from testing_pysift_utils import unpack_octave

In [81]:
sift = cv.xfeatures2d.SIFT_create()

In [82]:
dir_from = '/scratch/image_datasets/imagenet/'
dir_all   = '/scratch/image_datasets/1_for_learned_sift/all/'

In [83]:
files = listdir(dir_from)
files_nr = len(files)

In [84]:
files_nr

125

In [85]:
flickr_fail_image = cv.imread(dir_from + '/n00005787_71', 0) 

In [86]:
counter = 0


duplicate_threshold = 1  # the larger the threshold the closer patches will NOT be considered duplicates
random.shuffle(files)
for i in range(files_nr):

    try:
        image = cv.imread(dir_from + '/' + files[i], 0)
        if len(image.shape) != 2:
            continue
#         image = imageio.imread(dir_from + '/' + files[i])
    except Exception as e:
        print "Failed at", i, ":", e
        continue
    
    if image.shape == flickr_fail_image.shape and image[0].all() == flickr_fail_image[0].all() and image[1].all() == flickr_fail_image[1].all():
        print "Found Flickr fail image at", i
        continue

    keypoints, descriptors = sift.detectAndCompute(image, None)

    keypoints_unique = []
    descriptors_unique = []
    
    for j, keypoint in enumerate(keypoints):
        octave, layer, _ = unpack_octave(keypoint)
        if octave == 0 and layer == 1:
            duplicate = False
            for keypoint_unique in keypoints_unique:
                if abs(keypoint_unique.pt[0] - keypoint.pt[0]) < (keypoint.size / duplicate_threshold) and abs(keypoint_unique.pt[1] - keypoint.pt[1]) < (keypoint.size / duplicate_threshold):
                    duplicate = True
                    break
            if not duplicate:
                keypoints_unique.append(keypoint)
                descriptors_unique.append(descriptors[j])
                
    keypoints = keypoints_unique
    descriptors = np.array(descriptors_unique)
        
    for j, keypoint in enumerate(keypoints):
        octave, layer, _ = unpack_octave(keypoint)
        if octave == 0 and layer == 1:
            patch_centre_x = int(keypoint.pt[1])
            patch_centre_y = int(keypoint.pt[0])
            patch_diameter = int(2 * math.floor(keypoint.size / 2) + 1)  # rounding it to the nearest odd number
            patch_diameter *= 3

            if patch_centre_x - patch_diameter >= 0 and patch_centre_x + patch_diameter + 1 < image.shape[0] and patch_centre_y - patch_diameter >= 0 and patch_centre_y + patch_diameter + 1 < image.shape[1]:
                patch = image[patch_centre_x - patch_diameter: patch_centre_x + patch_diameter + 1,
                              patch_centre_y - patch_diameter: patch_centre_y + patch_diameter + 1]

#                 plt.imshow(patch, cmap="gray")
#                 plt.show()
                
                counter += 1
                
                                
                angle = keypoint.angle
                angle = (255. * angle) / 360
                angle = int(round(angle))
                
                descriptor = np.zeros(129, dtype=np.uint8)
                descriptor[: 128] = descriptors[j][:]
                descriptor[128] = angle
                
                
                cv.imwrite(dir_all + '/patches/patch' + str(counter).zfill(6) + '.bmp', patch)
                np.save(dir_all + '/descriptors_angles/descr_angle_' + str(counter).zfill(6) + '.npy', descriptor)
                


Failed at 0 : 'NoneType' object has no attribute 'shape'
(101, 128)


NameError: name 'jfkdsjkf' is not defined

In [70]:
counter

8617

In [65]:
counter

4526

In [ ]:
if image.shape == flickr_fail_image.shape and image[0].all() == flickr_fail_image[0].all() and image[1].all() == flickr_fail_image[1].all():
    print "it is a flickr fail image"

In [78]:
dir_all_patches = dir_all + '/patches/'
dir_all_descr = dir_all + '/descriptors_angles/'

dir_ready = '/scratch/image_datasets/1_for_learned_sift/ready/'

dir_train_patches = dir_ready + '/patches/train/class0/'
dir_validation_patches = dir_ready + '/patches/validation/class0/'
dir_test_patches = dir_ready + '/patches/test/class0/'

dir_train_descr = dir_ready + '/descriptors_angles/train/class0/'
dir_validation_descr = dir_ready + '/descriptors_angles/validation/class0/'
dir_test_descr = dir_ready + '/descriptors_angles/test/class0/'


# os.system('mkdir ' + directory_base + '/train')
# os.system('mkdir ' + directory_base + '/validation')
# os.system('mkdir ' + directory_base + '/test')

# os.system('mkdir ' + directory_train)
# os.system('mkdir ' + directory_validation)
# os.system('mkdir ' + directory_test)


files = os.listdir(dir_all_patches)
files_count = len(files)

random.shuffle(files)

cutoff1 = int(files_count * 0.8) # 40000 # int(files_count * 0.8)
cutoff2 = int(files_count * 0.9) # 40000 + 5000 # int(files_count * 0.9)
cutoff3 = files_count # 40000 + 5000 + 5000 # files_count


for i in range(cutoff1):
    
    patch_number = files[i][5:-4]
    os.system("mv " + dir_all_patches + files[i] + " " + dir_train_patches + files[i])
    os.system("mv " + dir_all_descr + "descr_angle_" + patch_number + '.npy' + " " + dir_train_descr + "descr_angle_" + patch_number + '.npy')
    
    
for i in range(cutoff1, cutoff2):
    patch_number = files[i][5:-4]
    os.system("mv " + dir_all_patches + files[i] + " " + dir_validation_patches + files[i])
    os.system("mv " + dir_all_descr + "descr_angle_" + patch_number + '.npy' + " " + dir_validation_descr + "descr_angle_" + patch_number + '.npy')
    
#     os.system("mv " + directory_all + "/" + files[i] + " " + directory_validation + "/" + files[i])
    
for i in range(cutoff2, cutoff3):
    patch_number = files[i][5:-4]
    os.system("mv " + dir_all_patches + files[i] + " " + dir_test_patches + files[i])
    os.system("mv " + dir_all_descr + "descr_angle_" + patch_number + '.npy' + " " + dir_test_descr + "descr_angle_" + patch_number + '.npy')
    
#     os.system("mv " + directory_all + "/" + files[i] + " " + directory_test + "/" + files[i])

In [53]:
#             x = random.randint(0, image.shape[0] - patch_size)
#             y = random.randint(0, image.shape[1] - patch_size)
#             patch = image[x : x + patch_size, y : y + patch_size, :]

#             scipy.misc.imsave(dir_to + '/patch_test_images_inpainting' + str(counter).zfill(5) + '_' +str(j) + '.bmp', patch)
#     #         patch.save(dir_to + '/patch_' + str(i).zfill(5) + '_' +str(j) + '.bmp')
#             print('Saving' + dir_to + '/patch_' + str(counter).zfill(5) + '_' +str(j) + '.bmp')

#     #         imageio.imwrite(dir_to + '/patch_' + str(i).zfill(5) + '_' +str(j) + '.bmp')    